# Description of the final project


This project is devoted to the question-answering task. You are going to work with the **DaNetQA** dataset.

DaNetQA is a question answering dataset for yes/no.

In [1]:
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 37.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6f94a3f1f5f0029e9d79cb7c5a59a5fd4ef84cbc41d1e4001aa6e36c63d63a20
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [35]:
! pip install mlflow --quiet
! nohup mlflow ui --port 5000 &

nohup: appending output to 'nohup.out'


In [3]:
import os
import sys
import warnings
import pprint

import json
import re
import string

from gensim.models import KeyedVectors
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
import numpy as np
import pandas as pd
from pymorphy2 import MorphAnalyzer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, confusion_matrix,
                             ConfusionMatrixDisplay, precision_score,
                             recall_score)
import torch
from torch.utils.data import (DataLoader, RandomSampler, random_split,
                              SequentialSampler, TensorDataset)
import tqdm
from transformers import (AdamW, BertForSequenceClassification,
                          BertModel, BertTokenizer, get_linear_schedule_with_warmup)

import nltk
nltk.download('punkt')

import mlflow
import mlflow.sklearn
import mlflow.pytorch

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 4.9 MB/s eta 0:00:00


In [5]:
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [36]:
MLFLOW_SERVER_URL = 'http://127.0.0.1:5000/'

warnings.filterwarnings("ignore")
np.random.seed(40)


# Data load and preprocess

In [7]:
! wget --no-check-certificate https://russiansuperglue.com/tasks/download/DaNetQA

! mv DaNetQA DaNetQA.zip

! unzip DaNetQA.zip

--2024-03-09 18:28:03--  https://russiansuperglue.com/tasks/download/DaNetQA
Resolving russiansuperglue.com (russiansuperglue.com)... 149.100.138.62
Connecting to russiansuperglue.com (russiansuperglue.com)|149.100.138.62|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 1293761 (1.2M) [application/zip]
Saving to: ‘DaNetQA’

DaNetQA             100%[===================>]   1.23M  --.-KB/s    in 0.1s    

2024-03-09 18:28:03 (11.1 MB/s) - ‘DaNetQA’ saved [1293761/1293761]

Archive:  DaNetQA.zip
   creating: DaNetQA/
  inflating: DaNetQA/train.jsonl     
   creating: __MACOSX/
   creating: __MACOSX/DaNetQA/
  inflating: __MACOSX/DaNetQA/._train.jsonl  
  inflating: DaNetQA/.DS_Store       
  inflating: __MACOSX/DaNetQA/._.DS_Store  
  inflating: DaNetQA/test.jsonl      
  inflating: __MACOSX/DaNetQA/._test.jsonl  
  inflating: DaNetQA/val.jsonl       
  inflating: __MACOSX/DaNetQA/._val.jsonl  
  inflating: __MACO

In [8]:
def load_data(path):
    rows = []
    with open(path) as f:
        for line in f:
            rows.append(json.loads(line))

    df = pd.DataFrame(rows, index=[row.pop("idx") for row in rows])

    return df

In [9]:
train = load_data("DaNetQA/train.jsonl")
val = load_data("DaNetQA/val.jsonl")

In [10]:
train.describe()

,question,passage,label
count,1749,1749,1749
unique,1399,1552,2
top,Были ли в австралии аборигены?,"НА́ТО, Организа́ция Североатланти́ческого дого...",True
freq,6,10,1061


In [11]:
val.describe()

,question,passage,label
count,821,821,821
unique,572,781,2
top,Есть ли такая рыба лосось?,"«Лу́нный за́говор» — ряд теорий заговора, цент...",True
freq,5,3,412


In [12]:
len((train["question"] + train["passage"]).unique())

1749

In [13]:
len((val["question"] + val["passage"]).unique())

819

In [14]:
question_passage = val["question"] + val["passage"] + val["label"].astype(str)

question_passage[question_passage.duplicated(keep=False)]

14     Едят ли яйца веганы?Вега́нство  — наиболее стр...
185    Едят ли яйца веганы?Вега́нство  — наиболее стр...
446    Была ли в россии чума?Эпидемия чумы 1654—1655 ...
552    Была ли в россии чума?Эпидемия чумы 1654—1655 ...
dtype: object

In [15]:
val = val.drop_duplicates(ignore_index = True)

In [16]:
train["label"].value_counts()

True     1061
False     688
Name: label, dtype: int64

In [17]:
val["label"].value_counts()

True     411
False    408
Name: label, dtype: int64

In [18]:
train = train.groupby("label").sample(n=688)

In [19]:
train["text"] = train["question"] + " " + train["passage"]

val["text"] = val["question"] + " " + val["passage"]


In [20]:
train_y = train["label"]
val_y = val["label"]

In [21]:
bert_tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [24]:
bert_tokens = bert_tokenizer(train["text"].to_list(), padding=True,truncation=True, return_tensors="pt")
bert_tokens_test = bert_tokenizer(val["text"].to_list(), padding=True,truncation=True, return_tensors="pt")

In [26]:
bert_dataset = TensorDataset(
    bert_tokens["input_ids"],
    bert_tokens["attention_mask"],
    torch.tensor(train_y.to_numpy(), dtype=torch.long)
)
bert_dataset_test = TensorDataset(
    bert_tokens_test["input_ids"],
    bert_tokens_test["attention_mask"],
    torch.tensor(val_y.to_numpy(), dtype=torch.long)
)

In [27]:
bert_dataset_train, bert_dataset_eval = random_split(bert_dataset, [0.9, 0.1])


Since there are no labels for test data in the data, we will consider the validation data to be a test data, and the training data will be divided into validation and training data.

In [29]:
BATCH_SIZE = 8
bert_dataloader_train = DataLoader(bert_dataset_train,
                                   sampler=RandomSampler(bert_dataset_train),
                                   batch_size=BATCH_SIZE)
bert_dataloader_eval = DataLoader(bert_dataset_eval,
                                   sampler=SequentialSampler(bert_dataset_eval),
                                   batch_size=BATCH_SIZE)
bert_dataloader_test = DataLoader(bert_dataset_test,
                                   sampler=SequentialSampler(bert_dataset_test),
                                   batch_size=BATCH_SIZE)

# Training, logging in Mlflow, selection of hyperparameters

As you can see, metrics for test values ​​are also calculated in train, but this is not used in any way in the selection of hyperparameters and training. I just need it to quickly view test metrics later

In [30]:
def train_eval(train_dataset, eval_dataset, lr=3e-5, eps=1e-8, epoch=3):
  DEVICE = torch.device("cuda")
  bert_classifier = BertForSequenceClassification.from_pretrained(
                          "bert-base-multilingual-cased",
                          num_labels=2,
                          output_attentions=False,
                          output_hidden_states=False
                        )
  bert_classifier.to(DEVICE)

  EPOCHS = epoch

  optimizer = torch.optim.AdamW(bert_classifier.parameters(), lr=lr, eps=eps)

  total_steps = len(bert_dataloader_train) * EPOCHS
  scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps = 0,
                                              num_training_steps = total_steps)

  torch.cuda.empty_cache()

  train_stats = []

  for epoch in range(EPOCHS):
      print(f"Epoch {epoch}")

      # Training
      print("Training")
      bert_classifier.train()

      train_losses = []
      for batch in tqdm.tqdm(bert_dataloader_train):
          input_ids = batch[0].to(DEVICE)
          attention_mask = batch[1].to(DEVICE)
          labels = batch[2].to(DEVICE)

          bert_classifier.zero_grad()

          output = bert_classifier(input_ids, attention_mask, labels=labels)
          loss = output.loss

          loss.backward()
          torch.nn.utils.clip_grad_norm_(bert_classifier.parameters(), 1.0)
          optimizer.step()
          scheduler.step()

          train_losses.append(loss.item())

      train_loss = np.array(train_losses).mean()
      print(f"Mean loss = {train_loss}")

      # Evaluation
      print("Evaluation")
      bert_classifier.eval()

      eval_losses = []
      eval_accuracies = []

      for batch in tqdm.tqdm(bert_dataloader_eval):
          input_ids = batch[0].to(DEVICE)
          attention_mask = batch[1].to(DEVICE)
          labels = batch[2].to(DEVICE)

          with torch.no_grad():
              output = bert_classifier(input_ids, attention_mask, labels=labels)

          eval_losses.append(loss.item())

          predictions = np.argmax(output.logits.detach().cpu().numpy(), axis=1).flatten()
          accuracy = accuracy_score(y_true=labels.detach().cpu().numpy(), y_pred=predictions)
          eval_accuracies.append(accuracy)

      eval_loss = np.array(eval_losses).mean()
      eval_accuracy = np.array(eval_accuracies).mean()

      print(f"Mean loss = {eval_loss}, mean accuracy = {eval_accuracy}")

      test_losses = []
      test_accuracies = []

      for batch in tqdm.tqdm(bert_dataloader_test):
          input_ids = batch[0].to(DEVICE)
          attention_mask = batch[1].to(DEVICE)
          labels = batch[2].to(DEVICE)

          with torch.no_grad():
              output = bert_classifier(input_ids, attention_mask, labels=labels)

          test_losses.append(loss.item())

          predictions = np.argmax(output.logits.detach().cpu().numpy(), axis=1).flatten()
          accuracy = accuracy_score(y_true=labels.detach().cpu().numpy(), y_pred=predictions)
          test_accuracies.append(accuracy)

      test_loss = np.array(test_losses).mean()
      test_accuracy = np.array(test_accuracies).mean()

      train_stats.append({
          "epoch": epoch,
          "train loss": train_loss,
          "eval loss": eval_loss,
          "eval accuracy": eval_accuracy,
      })
  return eval_accuracy, test_accuracy, bert_classifier

In [31]:
def objective(trial):
  with mlflow.start_run():
    epoch = trial.suggest_int("epoch", 2, 5)
    eps = trial.suggest_float('eps', 1e-8, 1e-4, log=True)
    lr = trial.suggest_float('lr', 1e-6, 1e-4, log=True)
    eval_accuracy, test_accuracy, bert_classifier = train_eval(bert_dataloader_train, bert_dataloader_eval, lr=lr, eps=eps, epoch=epoch)
    mlflow.log_param("epoch", epoch)
    mlflow.log_param("eps", eps)
    mlflow.log_param("lr", lr)
    mlflow.log_metric("eval accuracy", eval_accuracy)
    mlflow.log_metric("test accuracy", test_accuracy)
    components = {
        "model": bert_classifier,
        "tokenizer": bert_tokenizer,
        }
    mlflow.transformers.log_model(
        transformers_model=components,
        artifact_path='model',
        pip_requirements = ['torch==2.1.0', 'torchvision==0.16.0', 'jaxlib==0.4.23'],
        task="text-classification",
    )
  return eval_accuracy

In [37]:
mlflow.set_tracking_uri(MLFLOW_SERVER_URL)

experiment_name = 'experiment'
mlflow.set_experiment(experiment_name)

2024/03/09 18:35:01 INFO mlflow.tracking.fluent: Experiment with name 'experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/872536664353035649', creation_time=1710009301648, experiment_id='872536664353035649', last_update_time=1710009301648, lifecycle_stage='active', name='experiment', tags={}>

In [38]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5 ,show_progress_bar=True)

  0%|          | 0/5 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 0
Training



100%|██████████| 155/155 [01:52<00:00,  1.38it/s]


...


100%|██████████| 155/155 [01:59<00:00,  1.30it/s]


Mean loss = 0.4662489212328388
Evaluation



100%|██████████| 18/18 [00:04<00:00,  4.09it/s]


Mean loss = 0.3864991366863251, mean accuracy = 0.7291666666666666



100%|██████████| 103/103 [00:21<00:00,  4.90it/s]


# Results

In [39]:
from mlflow.tracking import MlflowClient
from pprint import pprint

In [40]:
client = MlflowClient(MLFLOW_SERVER_URL)
experiment = client.get_experiment_by_name(experiment_name)
run_info = client.search_runs(experiment.experiment_id)[0]

print(experiment)
print(run_info)

<Experiment: artifact_location='mlflow-artifacts:/872536664353035649', creation_time=1710009301648, experiment_id='872536664353035649', last_update_time=1710009301648, lifecycle_stage='active', name='experiment', tags={}>
<Run: data=<RunData: metrics={'eval accuracy': 0.7291666666666666, 'test accuracy': 0.5768608414239482}, params={'epoch': '4', 'eps': '1.2068824117439967e-07', 'lr': '6.74463305911642e-06'}, tags={'mlflow.log-model.history': '[{"run_id": "11b030207eee423dbe205d4675520ee3", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2024-03-09 19:29:46.925828", "flavors": '
                             '{"transformers": {"transformers_version": '
                             '"4.38.2", "code": null, "task": '
                             '"text-classification", "instance_type": '
                             '"TextClassificationPipeline", "framework": "pt", '
                             '"pipeline_model_type": '
     

In [48]:
reg_model_name = "bert-fine-tune"

client.create_registered_model(reg_model_name)

result = client.create_model_version(
    name=reg_model_name,
    source=f"{run_info.info.artifact_uri}/model",
    run_id=run_info.info.run_id
)

print(result)

2024/03/09 19:33:00 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: bert-fine-tune, version 1


<ModelVersion: aliases=[], creation_timestamp=1710012780021, current_stage='None', description='', last_updated_timestamp=1710012780021, name='bert-fine-tune', run_id='11b030207eee423dbe205d4675520ee3', run_link='', source='mlflow-artifacts:/872536664353035649/11b030207eee423dbe205d4675520ee3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>


Let's choose the best model based on the best validation sample metrics

In [50]:
run_infos = client.search_runs(experiment.experiment_id)
best_metric = 0
best_run_id = None
best_run_artifact_uri = None
for i in run_infos:
  staging_metrics = client.get_run(i.info.run_id).data.metrics['eval accuracy']
  if staging_metrics > best_metric:
    best_metric = staging_metrics
    best_run_id = i.info.run_id
    best_run_artifact_uri = i.info.artifact_uri

We put the best model in Production

In [53]:
result = client.create_model_version(
    name=reg_model_name,
    source=f"{best_run_artifact_uri}/model",
    run_id=best_run_id
)
client.transition_model_version_stage(
    name=reg_model_name,
    version=result.version,
    stage="Production"
)

2024/03/09 19:39:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: bert-fine-tune, version 2


<ModelVersion: aliases=[], creation_timestamp=1710013182557, current_stage='None', description='', last_updated_timestamp=1710013182557, name='bert-fine-tune', run_id='1a78d29dd3c84b578a106e7a05193a8a', run_link='', source='mlflow-artifacts:/872536664353035649/1a78d29dd3c84b578a106e7a05193a8a/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>


In [54]:
current_prod = [v for v in client.search_model_versions(f"name='{reg_model_name}'") if v.current_stage == 'Production'][-1]

In [55]:
current_prod

<ModelVersion: aliases=[], creation_timestamp=1710013182557, current_stage='Production', description='', last_updated_timestamp=1710013182576, name='bert-fine-tune', run_id='1a78d29dd3c84b578a106e7a05193a8a', run_link='', source='mlflow-artifacts:/872536664353035649/1a78d29dd3c84b578a106e7a05193a8a/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>


As you can see from the test sample, the model turned out to be overtrained. It was necessary to take a larger validation sample.

In [56]:
test_metric = client.get_run(best_run_id).data.metrics['test accuracy']

In [57]:
test_metric

0.5635113268608414


Save and check loading the model

In [59]:
import pickle

loaded_pipeline = mlflow.transformers.load_model(f"{best_run_artifact_uri}/model", return_type="pipeline")


with open('model.pickle', 'wb') as f:
  pickle.dump(loaded_pipeline, f)

with open('model.pickle', 'rb') as f:
  loaded_pipeline_loaded = pickle.load(f)

2024/03/09 19:44:27 INFO mlflow.store.artifact.artifact_repo: The progress bar can be disabled by setting the environment variable MLFLOW_ENABLE_ARTIFACTS_PROGRESS_BAR to false


2024/03/09 19:44:45 WARNING mlflow.transformers.model_io: Could not specify device parameter for this pipeline type.Falling back to loading the model with the default device.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Just checking, text have no senese

In [60]:
loaded_pipeline_loaded.predict(["Все будет хорошо"])

[{'label': 'LABEL_0', 'score': 0.6430174112319946}]